In [1]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import sys
sys.path.append('/work/users/d/d/ddinh/aaco/src')
from load_dataset import load_adni_data
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
import tensorflow as tf


2024-12-03 10:33:29.453664: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 10:33:29.474129: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 10:33:29.480189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 10:33:29.500262: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 10:33:31.503471: W tensorflow/compiler/tf2

## Data

In [2]:
def load_data(file_path):
    ds = load_adni_data(file_path=file_path)
    x = ds.x
    y = ds.y
    mask_nan = np.isnan(x)
    x[mask_nan] = 0
    
    mask_nan_y = np.isnan(y)
    y[mask_nan_y] = 0
    return x, y

train_x, train_y = load_data("/work/users/d/d/ddinh/aaco/input_data/train_data.npz")
val_x, val_y = load_data("/work/users/d/d/ddinh/aaco/input_data/val_data.npz")
test_x, test_y = load_data("/work/users/d/d/ddinh/aaco/input_data/test_data.npz")

num_ts = train_x.shape[1]

In [3]:
test_y[10]

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [4]:
def get_ts(x, y): 
    x_data = np.copy(x)
    y_data = np.copy(y)
    x_ts = []
    y_ts = []
    
    for i in range(num_ts):
        x_zero = np.zeros(x_data.shape)
        x_zero[:, :i+1] = x_data[:, :i+1]
        
        non_zero_mask = np.sum(y_data[:,i,:], axis=1) != 0
        x_filtered = np.transpose(x_zero[non_zero_mask,:,:], (0, 2, 1)).reshape(-1, x_data.shape[1] * x_data.shape[2])
        x_filtered = np.concatenate([x_filtered, np.repeat(i, x_filtered.shape[0])[:, None]], axis=1)
        x_ts.append(x_filtered)
        y_ts.append(y_data[non_zero_mask, i, :])
        
    x_ts = np.concatenate(x_ts, axis=0)
    y_ts = np.concatenate(y_ts, axis=0)
    
    
    return x_ts, y_ts

test_x_ts, test_y_ts = get_ts(test_x, test_y)


In [5]:
train_x_ts, train_y_ts = get_ts(train_x, train_y)
val_x_ts, val_y_ts = get_ts(val_x, val_y)

train_x_ts = np.concatenate([train_x_ts, val_x_ts], axis=0)
train_y_ts = np.concatenate([train_y_ts, val_y_ts], axis=0)

In [6]:
# load pt file 
import torch 
path = '/work/users/d/d/ddinh/aaco/results/dataset_adni_mlp_uncertainty.pt'
# path = '/work/users/d/d/ddinh/aaco/results/dataset_adnimlp_imputation_exclude_nan_mask_less_features.pt'
# path = '/work/users/d/d/ddinh/aaco/results/dataset_adni_mlp_progressively.pt'
data = torch.load(path)
x = data['X']
y = data['y']
mask = data['mask']
action = data['Action']

In [7]:
merge_action = []
count = 0
for i in range(action.shape[0]):
    if action[i, 0] == 1:
        merge_action.append(action[i])
        count += 1
    else:
        merge_action[count - 1] += action[i]
merge_action = np.array(merge_action)
merge_action = merge_action[:,:-1]

In [8]:
import tensorflow as tf
# model = tf.keras.models.load_model('/work/users/d/d/ddinh/aaco/models/mlp.keras')
model = tf.keras.models.load_model('/work/users/d/d/ddinh/aaco/models/mlp_uncertainty.keras')

In [9]:
merge_action.shape

(200, 48)

In [10]:
num_ts = y.shape[1]
y_pred = np.zeros(test_y.shape)
input_eval = []

for ts in range(num_ts):
    x_input = np.zeros(test_x.shape)
    mask_input = np.zeros(merge_action.shape)
    
    x_input[:,:ts+1,:] = np.copy(test_x[:,:ts+1,:])
    for k in range(4):
        # mask_input[:,k*ts:(k+1)*ts] = np.copy(merge_action[:,k*ts:(k+1)*ts])
        mask_input[:,k*num_ts:k*num_ts + ts + 1] = np.copy(merge_action[:,k*num_ts:k*num_ts + ts + 1])
    #  np.transpose(x_zero[non_zero_mask,:,:], (0, 2, 1)).reshape(-1, x_data.shape[1] * x_data.shape[2])    
    x_input = np.transpose(x_input, (0, 2, 1)).reshape(-1, test_x.shape[1] * test_x.shape[2])
    x_input = torch.Tensor(x_input)
    mask_input = torch.Tensor(mask_input)
    # mask_input = np.zeros((x_input.shape[0], 4 * num_ts))
    x_input = np.concatenate([x_input * mask_input, np.repeat(ts, x_input.shape[0])[:, None]], axis=1)
    
    mask_y = np.sum(test_y[:,ts], axis=1) != 0
    input_eval.append(x_input[mask_y])
    
    # per_mask_est = train_classifier_with_mask(mask_input.numpy())
    # pred = per_mask_est.predict_proba(x_input)
    # pred = np.array(pred)[:,0,:]
    
    pred = model.predict(x_input)
    y_pred[:,ts,:] = pred
    
input_eval = np.concatenate(input_eval, axis=0)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 938us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 916us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 830us/step


In [11]:
print(model.evaluate(input_eval, test_y_ts))

47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5695 - loss: 0.7650 - prc: 0.5556 - roc_auc: 0.7545   
[0.8008466362953186, 0.5464959740638733, 0.7404984831809998, 0.5446016192436218]


In [12]:
def evaluate_metrics(y_pred, y_true):
    auroc_metric = tf.keras.metrics.AUC(curve='ROC', multi_label=True, num_labels=3)
    auprc_metric = tf.keras.metrics.AUC(curve='PR', multi_label=True, num_labels=3)
    # auroc_metric = tf.keras.metrics.AUC(curve='ROC')
    # auprc_metric = tf.keras.metrics.AUC(curve='PR')
    
    all_preds = []
    all_labels = []

    for i in range(y_pred.shape[0]):
        for t in range(y_pred.shape[1]):  
            if not np.all(y_true[i, t] == 0):  
                all_preds.append(y_pred[i, t])
                all_labels.append(y_true[i, t])
            
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    # all_preds = np.argmax(all_preds, axis=1)
    # all_labels = np.argmax(all_labels, axis=1)
    
    auroc_metric.update_state(all_labels, all_preds)
    auprc_metric.update_state(all_labels, all_preds)
    
    roc_auc = auroc_metric.result().numpy()
    pr_auc = auprc_metric.result().numpy()
    
    return roc_auc, pr_auc

roc, pr = evaluate_metrics(y_pred, test_y)
print('ROC AUC:', roc)
print('PR AUC:', pr)

ROC AUC: 0.7404985
PR AUC: 0.5446016


In [27]:
# ROC AUC: 0.9001046
# PR AUC: 0.7775238

My version

In [ ]:
test_y[6]

In [ ]:
np.transpose(train_x, (0,2,1)).reshape(train_x.shape[0], -1)[:2]

In [ ]:
train_x[:2]

In [ ]:
dataset = load_adni_data()
x = dataset.x
y = dataset.y

mask_nan = np.isnan(x)
y_nan = np.isnan(y)

x[mask_nan] = 0
y[y_nan] = 0

In [ ]:
# import numpy as np

# # Assuming original shape of X is (n, 12, 4)
# def create_lagged_features(X, lag=2):
#     n, timesteps, modalities = X.shape
#     X_lagged = []

#     for t in range(lag, timesteps):
#         # Stack current timestep with lagged timesteps for each sample
#         features = X[:, t-lag:t+1, :].reshape(n, -1)  # Shape will be (n, (lag+1) * modalities)
#         X_lagged.append(features)
    
#     return np.concatenate(X_lagged, axis=0)  # Shape will be ((n * (timesteps - lag)), (lag+1) * modalities)

# # Generate lagged features
# lag = 2
# x_lagged = create_lagged_features(x, lag=lag)

# # Reshape targets accordingly to match the samples we have
# y_lagged = y[:, lag:].reshape(-1, 3)  # Remove initial timesteps for which we lack lag data
# y_lagged = np.argmax(y_lagged, axis=-1)  # Convert to class labels


In [ ]:
x_all = []
for item in x:
    x_all.append(item.flatten('F'))
x_all = np.array(x_all)

In [ ]:
# split dataset into train and test sets 80/20 with random seed 42
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_all, y, test_size=0.2, random_state=42)

In [ ]:
# load pt file 
import torch 
path = '/work/users/d/d/ddinh/aaco/results/dataset_adni_mlp_progressively.pt'
data = torch.load(path)
x = data['X']
y = data['y']
mask = data['mask']
action = data['Action']

In [23]:
merge_action = []
count = 0
for i in range(action.shape[0]):
    if action[i, 0] == 1:
        merge_action.append(action[i])
        count += 1
    else:
        merge_action[count - 1] += action[i]
merge_action = np.array(merge_action)
merge_action = merge_action[:,:-1]

In [24]:
merge_action.shape

(200, 48)

In [25]:
model = XGBClassifier()
model.load_model('/work/users/d/d/ddinh/aaco/models/adni_different_masking.model')

In [ ]:
plt.scatter(range(len(model.feature_importances_)), model.feature_importances_)

Error in callback <function flush_figures at 0x7f88afb33740> (for post_execute), with arguments args (),kwargs {}:


In [ ]:
x_val.shape

In [14]:
num_ts = y.shape[1]
y_pred = np.zeros(y_val.shape)
for ts in range(num_ts):
    x_input = np.zeros(x_val.shape)
    mask_input = np.zeros(merge_action.shape)
    
    for k in range(4):
        x_input[:,k*ts:(k+1)*ts] = np.copy(x_val[:,k*ts:(k+1)*ts])
        mask_input[:,k*ts:(k+1)*ts] = np.copy(merge_action[:,k*ts:(k+1)*ts])
    
    x_input = torch.Tensor(x_input)
    mask_input = torch.Tensor(mask_input)
    pred = model.predict_proba(torch.cat([x_input * mask_input, mask_input], -1))
    y_pred[:,ts,:] = pred

NameError: name 'y_val' is not defined

In [ ]:
y_pred.shape

In [ ]:
np.all(np.array([1,0,0]) == 0)

In [28]:
def evaluate_metrics(y_pred, y_true):
    auroc_metric = tf.keras.metrics.AUC(curve='ROC')
    auprc_metric = tf.keras.metrics.AUC(curve='PR')
    
    all_preds = []
    all_labels = []

    for i in range(y_pred.shape[0]):
        for t in range(y_pred.shape[1]):  
            if not np.all(y_true[i, t] == 0):  
                all_preds.append(y_pred[i, t])
                all_labels.append(y_true[i, t])
            
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    auroc_metric.update_state(all_labels, all_preds)
    auprc_metric.update_state(all_labels, all_preds)
    roc_auc = auroc_metric.result().numpy()
    pr_auc = auprc_metric.result().numpy()
    
    return roc_auc, pr_auc

roc, pr = evaluate_metrics(y_pred, y_val)
print('ROC AUC:', roc)
print('PR AUC:', pr)

NameError: name 'y_val' is not defined

In [57]:
test_x[0]

array([[-2.101955  ,  0.        , -0.03261582, -0.90718764],
       [-2.0911124 ,  0.        , -0.39757848, -1.6004739 ],
       [-2.485532  ,  0.        , -0.39757848, -1.6004739 ],
       [-2.485532  ,  0.        , -0.39757848, -1.6004739 ],
       [-2.6130905 ,  0.        , -0.39757848, -1.6004739 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]],
      dtype=float32)

In [59]:
test_x.shape

(200, 12, 4)

In [13]:
for i in range(test_x.shape[0]):
    for k in range(4):
        if test_x[i,0,k] == 0:
            merge_action[i, k * num_ts] = 0
            
PET = 1
MRI = 0.5

merge_action_reshaped = []
for ins in range(merge_action.shape[0]): 
    temp = []
    for k in range(4):
        temp.append(merge_action[ins,num_ts*k:num_ts*(k+1)])
    merge_action_reshaped.append(temp)
merge_action_reshaped = np.transpose(np.array(merge_action_reshaped), (0,2,1))

In [14]:
merge_action_reshaped = []
for ins in range(merge_action.shape[0]): 
    temp = []
    for k in range(4):
        temp.append(merge_action[ins,num_ts*k:num_ts*(k+1)])
    merge_action_reshaped.append(temp)
merge_action_reshaped = np.transpose(np.array(merge_action_reshaped), (0,2,1))

In [15]:
merge_action_reshaped.shape

(200, 12, 4)

In [16]:
total = 0
for i in range(4):
    print(merge_action[:, i*num_ts:(i+1)*num_ts].shape)
    if i <= 1:
        total += np.mean(np.sum(merge_action[:, i*num_ts:(i+1)*num_ts], 1)) * PET
    else:
        total += np.mean(np.sum(merge_action[:, i*num_ts:(i+1)*num_ts], 1)) * MRI
total

(200, 12)
(200, 12)
(200, 12)
(200, 12)


4.65500009059906

In [18]:
total = 0
count = 0
for j in range(merge_action.shape[0]):
    total_temp = 0
    
    for i in range(4):
        if i <= 1:
            total_temp += np.sum(merge_action[j, i*num_ts:(i+1)*num_ts]) * PET
        else:
            total_temp += np.sum(merge_action[j, i*num_ts:(i+1)*num_ts]) * MRI
            
    total += total_temp 
total / merge_action_reshaped.shape[0] 

4.655